In [1]:
from stmpy import Driver, Machine


class Quiz:
    def on_init(self):
        print("Init!")

    def ask_question(self):
        #print("What is 1+1?")
        self.mqtt_client.publish("question", "What is 1+1?")

    #def send_mqtt_tock(self):
        #print("Tock! {}".format(self.tocks))
        #self.tocks = self.tocks + 1
        #self.mqtt_client.publish("group16 tock", self.ticks)


# initial transition
t0 = {"source": "initial", "target": "waiting"}

t1 = {
    "trigger": "asked",
    "source": "waiting",
    "target": "question_asked",
}

t2 = {
    "trigger": "t",
    "source": "question_asked",
    "target": "waiting",

}
t3 = {
    "trigger": "buzz",
    "source": "question_asked",
    "target": "buzzer_pressed",
}
t4 = {
    "trigger": "next",
    "source": "buzzer_pressed",
    "target": "waiting",
}

waiting = {
    "name": "waiting"
}
question_asked = {
    "name": "question_asked",
    "entry": "start_timer('t', '20000')",
    "exit": "stop_timer('t')"
}
buzzer_pressed = {
    "name": "buzzer_pressed"
}

In [2]:
from threading import Thread

import paho.mqtt.client as mqtt


class MQTT_Client_1:
    def __init__(self):
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message

    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, msg):
        print("on_message(): topic: {}".format(msg.topic))
        
        if msg.topic == "asked":
            self.stm_driver.send("asked", "quiz")
            print(msg["msg"])
        elif msg.topic == "buzz":
            self.stm_driver.send("buzz", "quiz")
            print(msg["msg"])
        elif msg.topic == "next":
            self.stm_driver.send("next", "quiz")
            print(msg["msg"])
        else:
            self.stm_driver.send("message", "quiz")


    def start(self, broker, port):

        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)

        self.client.subscribe("question")
        self.client.subscribe("asked")
        self.client.subscribe("buzz")
        self.client.subscribe("next")


        try:
            # line below should not have the () after the function!
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()

In [3]:
# broker, port = 'iot.eclipse.org', 1883
#broker, port = "localhost", 1883
broker, port = "mqtt20.iik.ntnu.no", 1883

quiz = Quiz()
quiz_machine = Machine(transitions=[t0, t1, t2, t3, t4], obj=quiz, name="quiz", states=[waiting, question_asked, buzzer_pressed])
quiz.stm = quiz_machine

driver = Driver()
driver.add_machine(quiz_machine)

myclient = MQTT_Client_1()
quiz.mqtt_client = myclient.client
myclient.stm_driver = driver

driver.start()
myclient.start(broker, port)

Connecting to mqtt20.iik.ntnu.no:1883
on_connect(): Connection Accepted.


In [4]:
quiz.ask_question()

Machine quiz is in state waiting and received event message, but no transition with this event is declared!


on_message(): topic: question
on_message(): topic: asked
What is 1+1?
on_message(): topic: buzz
BZZZZZ
on_message(): topic: next
We think its 2
